<a href="https://colab.research.google.com/github/Programming-Communities/Assignment-Q2/blob/main/Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet google.genai


In [2]:
!pip install gTTS --quiet


In [3]:
!pip install playsound


  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=6996eb7016998ef6c9f5d0d76318671c67eaa6e1e3558810b6353719e93b322f
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [4]:
import google.generativeai as genai


In [6]:
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [10]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str ="gemini-2.0-flash-exp"


In [15]:
# This appears to be a file path and not Python code.  No correction needed.
# If you intend to use this in Python code, you'll likely need to put it in quotes:
video_file_name = "/content/WhatsApp Video 2024-12-28 at 11.05.03 AM.mp4"
#  (Note the double backslashes for escaping backslashes in Python strings)

In [16]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/4te04ep9edz


In [17]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
print(response.text)

```json
[
  {
    "timecode": "0:00",
    "caption": "A group of people are gathered under an orange and white tarp, some are drinking from small paper cups. There appears to be a wet street in the background"
  },
  {
    "timecode": "0:12",
     "caption": "A bearded man with glasses and a black turban is speaking to the camera “Bismillahir Rahmanir Rahim…”"
    },
   {
   "timecode": "0:16",
   "caption": "The man with the black turban is speaking: “Para Chinar ke mazloom Shia Musulman Kuram ki hamare Musulman jo guzishta dhai mah se jinka tal Para Chinar Road band hone ki wajah se…”"
    },
   {
    "timecode": "0:30",
     "caption":"The man is speaking: “Wahan par sakht mushkilat hain. Log mar rahe hain, log bhuke mar rahe hain, wahan pe davai nahi hai, 50, 55 se zyada bachche wahan par mar chuke hain….”"
   },
  {
    "timecode":"0:40",
    "caption":"The man continues speaking: “Aur aaj chautha roz yahan Press Club Lahore ke bahar hum dharna diye baithe hain. Hamari maahe-bahne

In [19]:

from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

# ... (Your code to extract text from the response) ...

tts = gTTS(text= "\"The man is speaking: “Wahan par sakht mushkilat hain. Log mar rahe hain, log bhuke mar rahe hain, wahan pe davai nahi hai, 50, 55 se zyada bachche wahan par mar chuke hain.\""

, lang='en')  # Create a gTTS object with the extracted text

# Save the audio to a file
with open("output.mp3", "wb") as f:
    for chunk in tts.stream():
        f.write(chunk)

# Play the audio using IPython.display Audio to ensure it works in Colab
# This is more reliable than playsound in Colab environments.
display(Audio("output.mp3", autoplay=True))